# Extension - RRT: basic finding example
* Path finding is using sample-based (RRT) methods rather than constrained optimization to find a collision free path
* Path finding is much easier if a final robot pose is given. We here use IK (formulated as KOMO problem) to first compute a final configuration 'qT'. (Path optimization does this jointly with optimizing the path.)
* Then we can pass the current state and qT to a bi-directional RRT to find a collision free path.
* Note that BotOp (the interface to sim/real) is opened only after we computed the motion. We simply pass the motion to be played by the sim/real.

In [1]:
import robotic as ry
import time
ry.params_clear()
ry.params_add({'rrt/stepsize':.1, 'rrt/verbose': 3}) #verbose=3 makes it very slow, and displays result, and verbose=4 waits keypress..

rrt = ry.PathFinder()
rrt.helloworld()

Hello, World!


In [ ]:
# # Create a new configuration
# C = ry.Config()

# C.addFile("/home/monke/Xrai/maze.g")

# # Add the moving "ego" frame with constraints on its motion
# C.addFrame("ego", "floorwalls") \
#     .setJoint(ry.JT.transXYPhi, [-1., 1., -1., 1., -3., 3.]) \
#     .setRelativePosition([0.2, 0.2, 0.4]) \
#     .setShape(ry.ST.ssBox, size=[0.05, 0.3, 0.05, 0.01]) \
#     .setColor([0, 1., 1.]) \
#     .setContact(1)

# C.addFrame("goal1") \
#     .setPosition(C.getFrame('ego').getPosition()+[0, -3, 0]) \
#     .setShape(ry.ST.ssBox, size=[0.05, 0.3, 0.05, 0.01]) \
#     .setColor([1, 0, 0]) \

# C.addFrame("goal2") \
#     .setPosition(C.getFrame('goal1').getPosition()+[4, 0, 0]) \
#     .setShape(ry.ST.ssBox, size=[0.05, 0.3, 0.05, 0.01]) \
#     .setColor([1, 0, 0]) \

# C.view()
# q0 = [0.2, 0.2, 0.4]
# qT = C.getFrame('goal1').getPosition()

# ry.params_clear()
# ry.params_add({'rrt/stepsize':.1, 'rrt/verbose': 0,'rrt/maxIters':500}) #verbose=3 makes it very slow, and displays result, and verbose=4 waits keypress..

# rrt = ry.PathFinder()
# rrt.setProblem(C, [q0], [qT])
# ret = rrt.solve()
# print(ret)
# path = ret.x

# # Visualize the solution path
# if ret.feasible:
#     for i in range(len(path)):
#         C.addFrame(f"path_{i}").setPosition(path[i]).setShape(ry.ST.sphere, [0.02])  # Add a sphere at each path point
#         C.view()
# else:
#     print("No valid path found.")

In [ ]:
# Create a new configuration
C = ry.Config()

C.addFile("/home/monke/Xrai/maze.g")

# Add the moving "ego" frame with constraints on its motion
C.addFrame("ego", "floorwalls") \
    .setJoint(ry.JT.transXYPhi, [-1., 1., -1., 1., -3., 3.]) \
    .setRelativePosition([0.2, 0.2, 0.4]) \
    .setShape(ry.ST.ssBox, size=[0.05, 0.3, 0.05, 0.01]) \
    .setColor([0, 1., 1.]) \
    .setContact(1)

C.addFrame("goal1") \
    .setPosition(C.getFrame('ego').getPosition()+[0, -3, 0]) \
    .setShape(ry.ST.ssBox, size=[0.05, 0.3, 0.05, 0.01]) \
    .setColor([1, 0, 0]) \

C.addFrame("goal2") \
    .setPosition(C.getFrame('goal1').getPosition()+[4, 0, 0]) \
    .setShape(ry.ST.ssBox, size=[0.05, 0.3, 0.05, 0.01]) \
    .setColor([1, 0, 0]) \

# C.view()
q0 = [0.2, 0.2, 0.4]
qT = C.getFrame('goal1').getPosition()

ry.params_clear()
ry.params_add({'rrt/stepsize':.05, 'rrt/verbose': 0,'rrt/maxIters':170}) #verbose=3 makes it very slow, and displays result, and verbose=4 waits keypress..

rrt = ry.PathFinder()
rrt.setProblem(C, [q0], [qT])
ret = rrt.solve()
ret_star = rrt.star_solve()
print(ret)
path = ret.x
star_path = ret_star.x
print(ret_star)




{ time: 0.003767, evals: 89, done: 1, feasible: 1, eq: -1, ineq: -1, sos: -1, f: -1 }
{ time: 0.010019, evals: 171, done: 1, feasible: 1, eq: -1, ineq: -1, sos: -1, f: -1 }


In [ ]:
# Create a new configuration
C = ry.Config()

C.addFile("/home/monke/Xrai/maze.g")

# Add the moving "ego" frame with constraints on its motion
C.addFrame("ego", "floorwalls") \
    .setJoint(ry.JT.transXYPhi, [-1., 1., -1., 1., -3., 3.]) \
    .setRelativePosition([0.2, 0.2, 0.4]) \
    .setShape(ry.ST.ssBox, size=[0.05, 0.3, 0.05, 0.01]) \
    .setColor([0, 1., 1.]) \
    .setContact(1)

C.addFrame("goal1") \
    .setPosition(C.getFrame('ego').getPosition()+[0, -3, 0]) \
    .setShape(ry.ST.ssBox, size=[0.05, 0.3, 0.05, 0.01]) \
    .setColor([1, 0, 0]) \

C.addFrame("goal2") \
    .setPosition(C.getFrame('goal1').getPosition()+[4, 0, 0]) \
    .setShape(ry.ST.ssBox, size=[0.05, 0.3, 0.05, 0.01]) \
    .setColor([1, 0, 0]) \

C.view()
# Visualize the solution path
for i in range(len(path)):
    C.addFrame(f"path_{i}").setPosition(path[i]).setShape(ry.ST.sphere, [0.02]).setColor([0,1,0])  # Add a sphere at each path point
    if i%5 == 0:
        C.view()
for i in range(len(star_path)):
    C.addFrame(f"path_{len(path)+i}").setPosition(star_path[i]).setShape(ry.ST.sphere, [0.02]).setColor([0,0,1])  # Add a sphere at each path point
    if i%5 == 0:
        C.view()

KeyboardInterrupt: 

In [ ]:
# import json

# C = ry.Config()

# C.addFile("/home/monke/Xrai/maze.g")

# # Add the moving "ego" frame with constraints on its motion
# C.addFrame("ego", "floorwalls") \
#     .setJoint(ry.JT.transXYPhi, [-1., 1., -1., 1., -3., 3.]) \
#     .setRelativePosition([0.2, 0.2, 0.4]) \
#     .setShape(ry.ST.ssBox, size=[0.05, 0.3, 0.05, 0.01]) \
#     .setColor([0, 1., 1.]) \
#     .setContact(1)

# C.addFrame("goal1") \
#     .setPosition(C.getFrame('ego').getPosition()+[0, -3, 0]) \
#     .setShape(ry.ST.ssBox, size=[0.05, 0.3, 0.05, 0.01]) \
#     .setColor([1, 0, 0]) \

# C.addFrame("goal2") \
#     .setPosition(C.getFrame('goal1').getPosition()+[4, 0, 0]) \
#     .setShape(ry.ST.ssBox, size=[0.05, 0.3, 0.05, 0.01]) \
#     .setColor([1, 0, 0]) \

# C.view()

# with open('path_data.json', 'r') as f:
#     path = json.load(f)
# for i in range(len(path)):
#     C.addFrame(f"path_{i}").setPosition(path[i]).setShape(ry.ST.sphere, [0.02])  # Add a sphere at each path point
#     C.view()